# Import Section
---

In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import Image, clear_output
from pydub import AudioSegment
import os

import math

import matplotlib.pyplot as plt
import numpy as np
from scipy.io.wavfile import read, write

# Audio Process Section
---
- Show the audio wav file fugure
- Processing the spliting wav file automatically

In [12]:
# auditok auto slice
import auditok

class audioFileP:  #father class
    def __init__(self, folder, filename):
        self.folder = folder
        self.filename = filename
        self.filepath = folder + '\\' + filename
         
    def create_tag_folder(self, tag_name):
        dir_path = os.path.join(os.getcwd(), tag_name)
        try:
            os.mkdir(dir_path)
        except OSError as error:
            print(error)
            print('skip create')
            
        print(os.getcwd())

class SplitWavAudionAutoV2(audioFileP):
    def __init__(self, folder, filename, tagfolder, show_plots):
        self.show_plots = show_plots
        audioFileP.__init__(self, folder, filename)
        print('Auto slice V2')
        self.rate, self.data = read(self.filepath)
        self.create_tag_folder('dataset')
        self.tag = 'dataset\\' + tagfolder
        self.create_tag_folder(self.tag)
        print("Sample rate: {} Hz".format(self.rate))
        print("Data type: {}".format(self.data.dtype))
    def btach_process(self):
        file_lists = os.listdir(self.folder)
        for i, v in enumerate(file_lists):
            #print(v)
            self.filename = v
            self.filepath = self.folder + '\\' + self.filename
            print(self.filepath)
            self.rate, self.data = read(self.filepath)
            self.auto_slice(self.get_auto_slice_array())
    
    def get_auto_slice_array(self):
        factor = 1                              #control window size
        win_size = self.rate                    #training data's format, 1(s)
        win_u = (int)((self.rate/10) * factor)  # windows is 0.1(s) 16000/1600
        s_r_factor = 0.5                        #search range factor, too long may find the next slice part, too short may not find the real max val.  
        
        ###create window's array for latter calculate###
        sum_array = np.zeros((len(self.data) - win_u + 1), dtype = int)  #use 0.1(s) windows size to capture/calculate
        data_abs = np.absolute(self.data)
        for idx, val in np.ndenumerate(sum_array):
            sum_array[idx[0]] = np.sum(data_abs[ idx[0] : (idx[0] + win_u) ])
        
        ###define parameters###
        mean_d_win = sum_array.mean()
        std_win    = sum_array.std()
        Max_win = sum_array.max()
        Min_win = sum_array.min()
        temp_max = 0
        temp_max_idx = 0
        start = 0
        end = 0
        slice_idx = 0
        slice_info = []
        
        ###find the slice's start&end idx and save all in slice_info array###
        for idx, val in np.ndenumerate(sum_array):
            if((val > mean_d_win + 1 * std_win) & (idx[0] > end)):  # first time trig  # only check over the previous lat cut point
                #print('begain:{}'.format(idx[0]))
                for i in range(idx[0], idx[0] + int(win_size*s_r_factor)):  # find the max window value in the range
                    if(i < len(sum_array) - 1):
                        if(temp_max < sum_array[i]):
                            temp_max = sum_array[i]
                            temp_max_idx = i
                            
                #print('max_loc:{} {}'.format(temp_max_idx,temp_max))        
                if((temp_max_idx - win_size/2 + 1) < 0):   # decide the start & end slice indexs
                    start = 0
                    end = win_size - 1
                elif((temp_max_idx + win_size/2) > (len(self.data) - 1)):
                    start = len(self.data) - win_size
                    end = len(self.data) - 1
                else:
                    start = (int)(temp_max_idx - win_size/2 + 1)
                    end = (int)(temp_max_idx + win_size/2)
                
                temp_max = 0
                slice_idx = slice_idx + 1
                slice_info.append((start, end))    

        return slice_info
        
    def auto_slice(self,slice_A):
        
        time = np.arange(0, len(self.data)) / self.rate
        plt.figure(figsize=(20, 10))
        
        plotA = plt.subplot(211)
        plotA.plot(time, self.data)
        plotA.set_ylabel("Amplitude")
        plotA.set_xlim(0, len(self.data) / self.rate)
        
        ###draw the slice boxes###
        max_data = self.data.max()    
        for i, v in enumerate(slice_A):
            if(i%2 == 1):
                plotA.vlines(v[0]/self.rate,-max_data-5000,max_data+5000,color="green")
                plotA.vlines(v[1]/self.rate,-max_data-5000,max_data+5000,color="green")
                plotA.hlines(-max_data-5000,v[0]/self.rate,v[1]/self.rate,color="green")
                plotA.hlines(max_data+5000,v[0]/self.rate,v[1]/self.rate,color="green")
            else:
                plotA.vlines(v[0]/self.rate,-max_data-5000,max_data+5000,color="red")
                plotA.vlines(v[1]/self.rate,-max_data-5000,max_data+5000,color="red")
                plotA.hlines(-max_data-5000,v[0]/self.rate,v[1]/self.rate,color="red")
                plotA.hlines(max_data+5000,v[0]/self.rate,v[1]/self.rate,color="red")
              
        
        ###save all the slice wavs###
        if self.show_plots:
            plts_len = len(slice_A)
            row = math.ceil(plts_len/2)
            col = 2
        
            fig, axs = plt.subplots(row, col, figsize=(15, 4 * row))
        
        for i, v in enumerate(slice_A):  #draw the all single slice wav
            if self.show_plots:
                time = np.arange(v[0], v[1]) / self.rate
                if(row>1):
                    axs[i//2, i%2].plot(time, self.data[v[0]:v[1]])
                    axs[i//2, i%2].set_ylabel("Amplitude")
                else:
                    axs[i].plot(time, self.data[v[0]:v[1]]) # when 1 row, the axs is 1-D
                    axs[i].set_ylabel("Amplitude")
            
            #save the slice data
            name = self.filename.split('_nohash')[0]
            split_fn = name + '_' + str(i) + '_nohash' 
            file_loc = os.getcwd()+ '\\' + self.tag + '\\' + split_fn
            if v[1] == len(self.data) - 1:
                end_s = v[1]
                start_s = v[0] - 1
            else:
                end_s = v[1] + 1
                start_s = v[0]
                
            write("{}.wav".format(file_loc), self.rate, self.data[start_s:end_s]) ##transfer from int32 to int16
            print("Region {i}: {s:.3f}s -- {e:.3f}s. Save as:{st}".format(i=i, s=v[0]/self.rate, e=v[1]/self.rate, st=file_loc))
            #print("Save as：{}.wav".format(i))
        
class SplitWavAudionAuto(audioFileP):
    def __init__(self, folder, filename, tagfolder):
        audioFileP.__init__(self, folder, filename)
        print('Can only process int16 file')
        self.audio_reg = auditok.load(self.filepath)
        self.tag = tagfolder
        self.create_tag_folder(tagfolder)
    def draw(self):
        self.audio_reg.plot()
    
    def auto_slice(self):
        audio_target = self.audio_reg.split_and_plot(
        min_dur=0.1,        # 聲音事件的最短長度
        max_dur=3,          # 聲音事件的最長長度
        max_silence=0.5,    # 聲音事件中無訊號最長長度
        energy_threshold=45, # 偵測聲音事件的能量門檻值
        analysis_window=0.01    
        )
        
        # 輸出分割聲音事件結果
        for i, r in enumerate(audio_target):
            # 輸出每段分割音訊的起始與結束時間點
            print("Region {i}: {r.meta.start:.3f}s -- {r.meta.end:.3f}s".format(i=i, r=r))
        
            # 撥放每段分割音訊
            # r.play(progress_bar=True)
        
            # 儲存每段分割音訊
            filename = r.save(os.getcwd()+ "\\" + self.tag + "\\" +"region_{meta.start:.3f}-{meta.end:.3f}.wav")
            print("儲存為：{}".format(filename))
            
        
class SplitWavAudioMubin(audioFileP):
    def __init__(self, folder, filename, tagfolder):
        audioFileP.__init__(self, folder, filename)
        print(self.filepath)
        self.audio = AudioSegment.from_wav(self.filepath)
        self.tag = tagfolder
        self.create_tag_folder(tagfolder)
    
    def get_duration(self):
        print(self.audio.duration_seconds)
        return self.audio.duration_seconds
    
    def single_split(self, from_sec, to_sec, split_filename):
        t1 = from_sec * 1000
        t2 = to_sec * 1000
        split_audio = self.audio[t1:t2]
        file_loc = os.getcwd()+ '\\' + self.tag + '\\' + split_filename
        split_audio.export(file_loc, format="wav")
        
    def multiple_split(self, sec_per_split, leap):
        total_secs = math.ceil(self.get_duration())
        for i in range(0, total_secs, sec_per_split+leap):
            split_fn = str(i) + '_' + self.filename
            self.single_split(i, i+sec_per_split, split_fn)
            print(str(i) + ' Done')
            

from scipy.io.wavfile import read                
class DrawWav():
    def __init__(self, folder, filename):
        self.folder = folder
        self.filename = filename
        self.filepath = folder + '\\' + filename
        self.rate, self.data = read(self.filepath)
        self.ch = 1 if self.data.size/self.data.shape[0]==1 else 2
    def print_Wavdata(self):
        #print(self.data)
        print("Sample rate: {} Hz".format(self.rate))
        print("Data type: {}".format(self.data.dtype))
        print("Data Seconds: {} s".format(self.data.shape[0]/self.rate))
        print("Number of channels: {}".format(self.ch))
        print("Data length: {}".format(len(self.data)))
    def stereoToMono(self):
        self.data = self.data.astype(float)
        w_data = self.data.sum(axis=1) / 2
        return w_data
        
    def plot_Wave(self):
        
        if(self.ch == 2):
            w_data = self.stereoToMono()
        else:
            w_data = self.data
        
        # Time data
        time = np.arange(0, len(w_data)) / self.rate
        plt.figure(figsize=(15, 5))

        # draw time domain
        plotA = plt.subplot(211)
        plotA.plot(time, w_data)
        plotA.set_ylabel("Amplitude")
        plotA.set_xlim(0, len(w_data) / self.rate)

        # draw frequency domain
        plotB = plt.subplot(212)
        plotB.specgram(w_data, NFFT=1024, Fs=self.rate, noverlap=900)
        plotB.set_ylabel("Frequency")
        plotB.set_xlabel("Time")

        plt.show()                   
                           

# Widgets Control Section
---

In [20]:
from ipyfilechooser import FileChooser
from os import walk
import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout
import shutil

class init_audio():
    def __init__(self):
        self.mypath = os.getcwd()
        self.dirpath,self.dirnames,self.filenames = self.getFirstLevel()
        
    def getFirstLevel(self): # get a list of folders 
        for (dirpath, dirnames, filenames) in walk(self.mypath):
            break
        return  dirpath, dirnames, filenames   
    
    def create_expanded_button(self, description, button_style):
        return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))

    def copy_allfiles(self, src_folder, dst_folder):
        copy_num = 0
        for file_name in os.listdir(src_folder):
            source = os.path.join(src_folder, file_name)
            destination = os.path.join(dst_folder, file_name)
            #print(source, destination)
            # copy only files
            if os.path.isfile(source):
                shutil.copy(source, destination)
                copy_num = copy_num + 1
                #print('copied', file_name)
        print("Copy finish, total {} files".format(copy_num))

    def interact_block_audio(self):
    
        dirnames = self.dirnames
        choose_folder = widgets.Dropdown(options=dirnames)
        choose_file = widgets.Dropdown(options=os.listdir(choose_folder.value))
 
        def update_choose_file(*args):   # Updates the choose_file options based on choose_folder value
            choose_file.options = os.listdir(choose_folder.value)
            print(choose_folder.value)
        choose_folder.observe(update_choose_file, 'value')   # Tie the choose_file options to choose_folder value
    
        def show_WorkArea(fdir, file):
            folder_loc = os.getcwd()+'\\'+fdir
            print('choosed folder:{}'.format(folder_loc))
            print('choosed file:{}'.format(file))
            
            button_a = self.create_expanded_button('Run Wav Plot', 'success')
            button_b = self.create_expanded_button('Slice Wav', 'info')
            button_c = self.create_expanded_button('Slice Wav Auto', 'warning')
            button_d = self.create_expanded_button('Copy to where(for training use)', 'info')
            button_e = self.create_expanded_button('Footer', 'success')

            output = widgets.Output() # buttom interact
            display( # show the buttom and output
                    AppLayout(header=None, left_sidebar=button_a, center=button_b, right_sidebar=button_c, footer=button_d,
                              pane_widths=[3, 3, 3], pane_heights=['40px', '40px', '40px']), output) 
            
            def on_button_clicked_eventA(b):
                with output:
                    clear_output()
                    draw_wav = DrawWav(folder_loc, file)
                    draw_wav.print_Wavdata()
                    draw_wav.plot_Wave()     
            button_a.on_click(on_button_clicked_eventA)
            
            
            def on_button_clicked_eventB(b):
                with output:
                    clear_output()
                    print('Please set the slice!!')
                                 
                    def act_split_wav(sec_per_split, leap, tag_name):
                        split_wav = SplitWavAudioMubin(folder_loc, file, tag_name)
                        split_wav.multiple_split(sec_per_split , leap)
                        return (sec_per_split, leap, tag_name)
                    
                    evt = interact_manual(act_split_wav, sec_per_split = widgets.IntSlider(min=0, max=10, step=1, value=10, description='Seconds/Slice'), 
                                        leap = widgets.IntSlider(min=0, max=10, step=1, value=1, description='Gap'),
                                        tag_name = widgets.Text(value='test_slice', description='Label'))
                    evt.widget.children[3].description = 'Start Slice'  #because there are 3 parameter of the evt
                    evt.widget.children[3].button_style = 'success'
            button_b.on_click(on_button_clicked_eventB)
            
            def on_button_clicked_eventC(b):
                with output:
                    clear_output()
                    
                    def act_auto_split(tag_name, batch_en, show_plots):
                        if not batch_en:
                            split_auto_v2 = SplitWavAudionAutoV2(folder_loc, file, tag_name, show_plots) 
                            split_auto_v2.auto_slice(split_auto_v2.get_auto_slice_array())
                        else:
                            print('auto slice batch')
                            split_auto_v2 = SplitWavAudionAutoV2(folder_loc, file, tag_name, show_plots)
                            split_auto_v2.btach_process()
                   
                    evt = interact_manual(act_auto_split, 
                                        tag_name = widgets.Text(value='test_slice', description='Label'),
                                        batch_en = widgets.Checkbox(value=True, disabled=False, indent=False, description='Batch Enable'),
                                        show_plots = widgets.Checkbox(value=False, disabled=False, indent=False, description='Show Plots'))
                    evt.widget.children[3].description = 'Start Slice'  #because there are 3 parameter of the evt
                    evt.widget.children[3].button_style = 'success'
                
                              
            button_c.on_click(on_button_clicked_eventC)
            
            def on_button_clicked_eventD(b):
                with output:
                    clear_output()
                    
                    path_fc = os.path.abspath('dataset') ##The processed data are in /dataset
                    fc = FileChooser(path_fc)
                    fc.show_only_dirs = True
                    fc.title = f"<b><font color='lightgreen'><font size=4>Choose train label folder.</b>"
                    display(fc)
                    
                    path_f_copy = os.path.abspath('..')  ##The train program is in ../ML_kws_tflu
                    path_f_copy = os.path.join(path_f_copy, 'ML_kws_tflu', 'tmp', 'speech_dataset')
                    
                    #if not os.path.isdir(path_f_copy):  ##check if the folder exists or not, if not creat it.
                    #    os.makedirs(path_f_copy)
                    #    print("<Create a new folder: {}>".format(path_f_copy))
                    #f_copy = FileChooser(path_f_copy)
                    #f_copy.show_only_dirs = True
                    #f_copy.title = f"<b><font color='lightgreen'><font size=4>Copy to which train dataset.</b>"
                    #display( f_copy)
                  
                    
                    def act_copy_folder():
                        print("Selected folder: {}".format(fc.selected_path))
                        copied_folder_name = fc.selected_path.split('\\')[-1]
                        copied_dst = os.path.join(path_f_copy, copied_folder_name)
                        if not os.path.isdir(copied_dst):  ##check if the folder exists or not, if not creat it.
                            os.makedirs(copied_dst)
                        
                        print("Copy to: {}".format(path_f_copy))
                        self.copy_allfiles(fc.selected_path, copied_dst)
                   
                    evt = interact_manual(act_copy_folder, 
                                        #folder_loc = widgets.Textarea(value='C:\\Users\\USERNAME\\MICRO_ML\\ML_kws_tflu\\tmp\\speech_dataset', description='Copy to:', font_size=75)  
                                         )
                    evt.widget.children[0].description = 'Start Copy'  #because there are 3 parameter of the evt
                    evt.widget.children[0].button_style = 'success'
                        
            button_d.on_click(on_button_clicked_eventD)
            
    
        _ = interact(show_WorkArea, fdir=choose_folder, file=choose_file)
       
    




# Run Section
- The detail description is here [meaning](#id-control-intro)

In [21]:
audio_proj = init_audio()
audio_proj.interact_block_audio()

interactive(children=(Dropdown(description='fdir', options=('.ipynb_checkpoints', 'dataset', 'one'), value='.i…

<a id="id-control-intro"></a>
# Control Introduction
---

- `fdir`: the folders in `ML_audio_aq` folder, you can choose any one of them.
- `file`: the files in the choosen folder, you can choose any one of them.
- `Run Wav Plot`: show the *.wav's figure.
- `Slice Wav`: manually slice the single wav file(old).
- `Slice Wav Auto`: smartly slice the single or whole folder's wav files.

- In `Slice Wav Auto`:
    1. Label is the folder's name which save all the sliced wav files. You should follow the training's label(answer).
    2. `Btach Enable`: slice the whole folder's wav files.
    3. `Show Plots`: show each sliced small plots.
    4. `Start Slice`: you can click this buttom after all setting is done.
    5. (recommend)(will update) The output `Label` folder is saved in `dataset`. You can copy/move the folders to training step's train_data_folder, for example: in `ML_kws_tflu\tmp\speech_dataset`